# Import libraries

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l1, l2

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
!nvidia-smi

# Load dataset

In [ ]:
trainingset = np.loadtxt('dataset/fer2013_training_onehot.csv', delimiter=',')
testingset = np.loadtxt('dataset/fer2013_publictest_onehot.csv', delimiter=',')

trainingset

In [ ]:
testingset.shape

# Split data

In [ ]:
n_inputs = 48*48
n_classes = 7
img_dim = 48

x_training = trainingset[:, 0:n_inputs]
y_training = trainingset[:, n_inputs:n_inputs + n_classes]

x_testing = testingset[:, 0:n_inputs]
y_testing = testingset[:, n_inputs:n_inputs + n_classes]

x_training = x_training.reshape(x_training.shape[0], 48, 48)
#x_training.shape
x_training = np.expand_dims(x_training, axis=3)

x_testing = x_testing.reshape(x_testing.shape[0], 48, 48)
x_testing = np.expand_dims(x_testing, axis=3)

In [ ]:
def get_emotion(ohv):
    if ohv.shape[0] == 1:
        indx = ohv[0]
    else:
        indx = np.argmax(ohv)
        
    if indx == 0:
        return 'angry'
    elif indx == 1:
        return 'disgust'
    elif indx == 2:
        return 'fear'
    elif indx == 3:
        return 'happy'
    elif indx == 4:
        return 'sad'
    elif indx == 5:
        return 'surprise'
    elif indx == 6:
        return 'neutral'

In [ ]:
print(x_training.shape)
print(y_training.shape)

print(x_testing.shape)
print(y_testing.shape)

sample = x_training[5, :]
sample = sample.reshape(48, 48)

plt.imshow(sample, cmap='gray')
plt.show()
print('facial emotion: %s' % get_emotion(y_training[5, :]))

# Define model: a model in karas is a sequence of layers!

In [ ]:
model = Sequential()

In [ ]:
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape = (48, 48, 1)))

In [ ]:
model.add(Conv2D(64, (3, 3), activation='relu'))

In [ ]:
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

In [ ]:
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))

In [ ]:
model.add(Conv2D(128, (3, 3), activation='relu'))

In [ ]:
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))

In [ ]:
model.add(Conv2D(256, (3, 3), activation='relu'))

In [ ]:
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

In [ ]:
model.add(Flatten())
model.add(Dense(1024, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='softmax'))

# Compile model

In [ ]:
opt = Adam(lr=0.0001, decay=10e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

# Fit model

In [ ]:
batch_size = 128
n_epoches = 500
early_stopping = EarlyStopping(monitor='val_loss', patience=20)

model.fit(x_training, y_training, batch_size=batch_size, epochs=n_epoches,
          validation_data=(x_testing, y_testing), shuffle=True,
          callbacks=[early_stopping])

# Evaluate model

In [ ]:
scores = model.evaluate(x_testing, y_testing)
print('%s: %.2f%%'% (model.metrics_names[1], scores[1]*100))

In [ ]:
model.save('fer2013.h5')
model.save_weights('fer2013_weights.h5')

# Load trained model

In [ ]:
model.load_weights('fer2013_weights.h5')

In [ ]:
scores = model.evaluate(x_testing, y_testing)
print('%s: %.2f%%'% (model.metrics_names[1], scores[1]*100))

***
# Test trained model

In [ ]:
# 2163
img_indx = np.uint32(np.random.rand()*(testingset.shape[0] - 1))
sample = x_testing[img_indx, :]
sample = sample.reshape(48, 48)

pred_cls = model.predict_classes(sample.reshape(1, 48, 48, 1))

plt.imshow(sample, cmap='gray')
plt.show()
print('> testing image index: %d\n> true emotion: %s\n> predicted emotion: %s' % (img_indx, get_emotion(y_testing[img_indx, :]), get_emotion(pred_cls)))

***
# Partial accuracy

In [ ]:
for emo_indx in range(0, n_classes):
    data_for_class = testingset[testingset[:, 2304 + emo_indx] == 1]
    x_data = data_for_class[:, 0:2304]
    x_data = x_data.reshape(x_data.shape[0], 48, 48)
    x_data = np.expand_dims(x_data, axis=3)

    y_data = data_for_class[:, 2304:2304 + n_classes]

    scores = model.evaluate(x_data, y_data, batch_size=32, verbose=0)
    print('> Accuracy %.2f%% for <%s>'% (scores[1]*100, get_emotion(np.array([emo_indx]))))